## Compressive sensing

Quantum state tomography aims to reconstruct the density matrix of a quantum system from measurement data. Traditional approaches rely on informationally complete measurements, which require a large number of different measurement settings, making the process resource-intensive. Compressive sensing offers an alternative that allows for reconstruction using fewer measurements by leveraging the sparsity of quantum states in a suitable basis.

### **Overview of the Code**

This code implements a compressive sensing approach to quantum state tomography, where the density matrix is reconstructed from incomplete measurement data using convex optimization techniques.

#### **Key Steps in the Code**

1. **Generating Pauli Measurement Bases** (`string2paulibasis`)
   - Given a list of binary strings, this function constructs tensor products of Pauli matrices to form measurement bases for multi-qubit systems.
   - The Pauli matrices are represented as:
     - X = [[1, 1], [1, -1]] / sqrt(2)
     - Y = [[1, 1], [i, -i]] / sqrt(2)
     - Z = [[1, 0], [0, 1]]
   - The function iterates through the list of indices and constructs the tensor product of these matrices accordingly.

2. **Constructing Measurement Operators** (`base2operator`)
   - Converts the measurement bases into a vectorized representation of POVM elements.
   - The function constructs projectors corresponding to the basis vectors and normalizes them.
   - If required, it reshapes the POVMs into a vectorized form to align with the linear tomography framework.

3. **Generating a Random Quantum State** (`density_matrix`)
   - Samples a Haar-random pure state and constructs its density matrix as `|ψ⟩⟨ψ|`.

4. **Reconstructing the Density Matrix Using Compressive Sensing** (`compressed_sensing`)
   - Given measured probabilities and the POVM matrix, the density matrix is estimated using convex optimization.
   - The constraints ensure:
     - The reconstructed matrix is positive semidefinite.
     - The trace is normalized to 1.
     - The measured probabilities satisfy a bound determined by the measurement noise.
   - The objective function minimizes the nuclear norm (sum of singular values) to promote low-rank solutions.

### **Mathematical Formulation**

The probability of measuring a specific outcome is given by Born’s rule: $P(E_i | \rho) = \text{Tr}(E_i \rho)$
where $E_i$ are the measurement operators (POVM elements) and $\rho$ is the density matrix.

In a standard quantum state tomography setup, we collect a set of probabilities $p_i$ and solve the linear system:
$A^\dagger \vec{\rho} = \vec{p}$
where $A = (\vec{E}_1, \vec{E}_2, \dots, \vec{E}_n )$ contains vectorized POVM elements. Since our measurements are not informationally complete, we use compressive sensing techniques to estimate $\rho$ by solving:
$$ \min_{\rho} \| \rho \|_{\ast}, \quad \text{subject to } \| A^\dagger \vec{\rho} - \vec{p} \| \leq \epsilon, \quad \rho\geq 0,  \quad tr(\rho)=1,$$
where $\| \rho \|_{\ast}$ is the nuclear norm (sum of singular values) and $\epsilon$ accounts for noise in the measurements.



In [1]:
import numpy as np
import scipy as sp
import cvxpy as cp

(CVXPY) Mar 21 12:05:25 PM: Encountered unexpected exception importing solver OSQP:
ImportError('DLL load failed while importing qdldl: No se puede encontrar el módulo especificado.')


In [2]:
def string2paulibasis(idx):
    """
    Given a list of binary strings, gives the Pauli bases. 
    PARAMETERS:
        idx: list. List of strings of the form "012..."
    RETURNS:
        bases: (2**n_qubits, 2**n_qubits, len(idx)) complex array. It contains
              the bases associated to the Pauli strings in idx.
    """
    # define the one-qubit bases
    X = np.array([[1., 1.], [1., -1.]], dtype=complex)/np.sqrt(2)
    Y = np.array([[1, 1], [1.*1j, -1.*1j]], dtype=complex)/np.sqrt(2) 
    Z = np.array([[1, 0], [0, 1]], dtype=complex)
    sigma = np.stack([X, Y, Z], axis=-1)
    n_qubits = len(idx[0])
    bases = np.zeros((2**n_qubits, 2**n_qubits, len(idx)), dtype="complex")
    # calculate all the combinations of tensor products
    for i in range(len(idx)):
        basis_i = 1
        for k in range(n_qubits):
            basis_i = np.kron(basis_i, sigma[:, :, int(idx[i][k])])
        bases[:, :, i] = basis_i
    return bases


def base2operator(matrix, vec=False):
    """
    Dada una lista de bases construye la representacion vectorial de el POVM
    IN
        matrix: array di x di x N. Bases.
    OUT
        vectores: array di x N. Agrupa todos los observables
    """
    dim = matrix.shape
    if len(dim) == 2:
        A = np.zeros((dim[0], dim[0], dim[1]), dtype="complex")
        for k in range(dim[1]):
            evector = matrix[:, k].reshape(-1, 1)
            projector = np.kron(evector.conj().T, evector)
            A[:, :, k] = projector
    else:
        A = np.zeros((dim[0], dim[0], dim[2]*dim[1]), dtype="complex")
        for j in range(dim[2]):
            for k in range(dim[1]):
                evector = matrix[:, k, j].reshape(-1, 1)
                projector = np.kron(evector.conj().T, evector)
                A[:, :, dim[0]*j + k] = projector/int(matrix.shape[-1])
    if vec:
        A = A.reshape(dim[0]**2, -1, order="F")
    
    return A


def density_matrix(di):
    "Density matrix for a random pure state of dimension di"

    psi = sp.stats.unitary_group.rvs(di)[:, 0]
    dens = np.outer(psi.conj(), psi)
    
    return dens



def compressed_sensing(prob, A, epsilon):
    """
    Gets the density matrix using the matrix of POVMs A and the measured
    probabilities using compressive sensing.
    IN
        prob: array (di*N, ). probabilities
        A: array (di, di, N). Matrix with all the POVM elements
    OUT
        dens: array (di, di). density matrix.
    """
    # optimizacion convexa
    di2 = A.shape[0]
    di = int(np.sqrt(di2))
    de = cp.Variable((di, di), hermitian=True)
    constraints = [de >> 0]
    constraints += [cp.trace(de) == 1]
    constraints += [cp.norm(A.conj().T@cp.reshape(de, (di2, 1), order="F")
                   - prob.reshape(-1, 1)) <= epsilon]
    objective = cp.Minimize(cp.normNuc(de))
    problema = cp.Problem(objective, constraints)
    result = problema.solve()


    return de.value/np.trace(de.value)

In [3]:
# set the number of qubits, the Pauli bases to measure and the shots
n_qubits = 4
di = 2**n_qubits
idx = ["0000", "1111", "2222"]
n_shots = 10**6

In [ ]:
# definition state and others
dens = density_matrix(di)
paulis = string2paulibasis(idx)
# vectorized version of the POVM
vec_paulis = base2operator(paulis, True)
# we calculate the exact probabilities and the ones with shot noise
probs = np.abs(vec_paulis.conj().T@dens.reshape(-1, 1, order="F"))
exp_probs = np.random.multinomial(n_shots, probs.flatten())/n_shots

In [5]:
# this is the tomography with a finite number of shots
epsilon_shots = np.sqrt(np.sum(exp_probs*(1 - exp_probs))/n_shots)
dens_estimated_shots = compressed_sensing(exp_probs, vec_paulis, epsilon_shots)
# this is the tomography using an infinite number of shots
epsilon = 0
dens_estimated = compressed_sensing(probs, vec_paulis, epsilon)

print("fidelity: ", np.round(np.abs(np.trace(dens_estimated@dens)), 3), 
      "fidelity_shots: ", np.round(np.abs(np.trace(dens_estimated_shots@dens)), 3))

fidelity:  0.259 fidelity_shots:  0.259
